In [ ]:
# setup notebook if it is run on Google Colab, cwd = notebook file location
try:
    # change notebook_path if this notebook is in a different subfolder of Google Drive
    notebook_path = "Projects/QuantumFlow/notebooks"

    import os
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir("/content/gdrive/My Drive/" + notebook_path)

    %tensorflow_version 2.x
    !pip install -q ruamel.yaml

    %load_ext tensorboard
except:
    pass

# imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import ipywidgets as widgets
from IPython.display import display

# setup paths and variables for shared code (../quantumflow) and data (../data)
import sys
sys.path.append('../')
data_dir = "../data"

# import shared code, must run 0_create_shared_project_files.ipynb first!
from quantumflow.utils import load_hyperparameters, train, build_model, QFDataset

In [ ]:
experiment = 'models'

base_dir = os.path.join(data_dir, experiment)
figures_dir = os.path.join(data_dir, 'models/figures')
if not os.path.exists(figures_dir): os.makedirs(figures_dir)

figsize = (10, 3)
dpi=None

In [ ]:
%tensorboard --logdir=$base_dir

In [ ]:
models = ['cnn',
          'resnet',
          'resnet_vW_N2',
          'models_large/resnet_vW_N2_1000',
          'models_large/resnet_vW_N2_10000',
          'models_large/resnet_vW_N2_100000']

model_params = {model: load_hyperparameters(os.path.join(base_dir, 'models_large'*model.startswith('models_large') , "hyperparams.config"), run_name=model.replace('models_large/', '')) for model in models}

In [ ]:
{model: params['model_kwargs']['l2_regularisation'] for model, params in model_params.items()}

In [ ]:
from tensorflow.python.summary.summary_iterator import summary_iterator
import pandas as pd 

def load_summaries(event_dir):
    summaries = {}

    assert os.path.exists(event_dir), event_dir

    event_files = [os.path.join(event_dir, file) for file in os.listdir(event_dir) if '.tfevents' in file]

    assert len(event_files) > 0, event_dir

    def create_or_append(tag, step, wall_time, keys, values):
        try:
            if step not in summaries[tag]['step']:
                summaries[tag]['step'].append(step)
                #summaries[tag]['wall_time'].append(wall_time)
                if isinstance(keys, list):
                    for key, value in zip(keys, values):
                        summaries[tag][key].append(value)
                else:
                    summaries[tag][keys].append(values)
        except KeyError:
            summaries[tag] = {'step': [step]}#, 'wall_time': [wall_time]}
            if isinstance(keys, list):
                for key, value in zip(keys, values):
                    summaries[tag][key] = [value]
            else:
                summaries[tag][keys] = [values]
            
    for event_file in event_files:
        for summary in summary_iterator(event_file):
            if summary.summary.value.__len__() == 0: continue   

            for entry in summary.summary.value:
                if entry.tag == 'keras':
                    continue # model config
                elif 'bias' in entry.tag or 'kernel' in entry.tag: 
                    if 'image' in entry.tag:
                        create_or_append('image/' + entry.tag.replace('image/', ''), summary.step, summary.wall_time, entry.tag.replace('image/', ''), entry.image.encoded_image_string)
                    else:
                        continue #histograms
                else: # metrics
                    create_or_append(entry.tag, summary.step, summary.wall_time, 'simple_value', entry.simple_value)
                
    for key in summaries.keys():
        summaries[key] = pd.DataFrame(data=summaries[key]).set_index('step')
    return summaries

In [ ]:
train_tfevents_dir = {model: os.path.join(base_dir, model, 'train') for model in models}
model_summaries = {model: load_summaries(tfevents_dir) for model, tfevents_dir in train_tfevents_dir.items()}
model_summaries['resnet'].keys()

In [ ]:
model_labels = {model: model.replace('models_large/', '').replace('_1', '').replace('0', '') + ' (M: {}, {} epochs)'.format(100 if model_params[model]['fit_kwargs']['epochs'] == 100000 else int(30000000/model_params[model]['fit_kwargs']['epochs']), model_params[model]['fit_kwargs']['epochs']) for model in models}

In [ ]:
plt.figure(figsize=figsize, dpi=dpi)
for model, summary in model_summaries.items():
    plt.plot(summary['learning_rate']['simple_value'].index/model_params[model]['fit_kwargs']['epochs']*(100000 if model_params[model]['fit_kwargs']['epochs'] == 100000 else 300000), summary['learning_rate'], label=model_labels[model])
plt.grid()
plt.yscale('log')
plt.legend()
plt.ylabel('learning rate')
plt.xlabel('training step')
plt.savefig(os.path.join(figures_dir, 'learning_rate.eps'), format='eps', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=figsize, dpi=dpi)
for model, summary in model_summaries.items():
    plt.plot(summary['learning_rate']['simple_value'].index/model_params[model]['fit_kwargs']['epochs'], summary['loss'], label=model_labels[model])
plt.yscale('log')
plt.grid()
plt.legend()
plt.ylabel('loss')
plt.xlabel('training progress')
plt.savefig(os.path.join(figures_dir, 'loss.eps'), format='eps', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=figsize, dpi=dpi)
for model, summary in model_summaries.items():
    plt.plot(summary['learning_rate']['simple_value'].index/model_params[model]['fit_kwargs']['epochs'], summary['mean_absolute_error/kinetic_energy'], label=model_labels[model])
plt.yscale('log')
plt.grid()
plt.legend()
plt.ylabel('mean absolute error / hartree')
plt.xlabel('training progress')
plt.savefig(os.path.join(figures_dir, 'kinetic_energy.eps'), format='eps', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=figsize, dpi=dpi)
for model, summary in model_summaries.items():
    plt.plot(summary['learning_rate']['simple_value'].index/model_params[model]['fit_kwargs']['epochs'], summary['mean_absolute_error/derivative'], label=model_labels[model])
plt.yscale('log')
plt.grid()
plt.legend()
plt.ylabel('mean absolute error / hartree')
plt.xlabel('training progress')
plt.savefig(os.path.join(figures_dir, 'derivative.eps'), format='eps', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=figsize, dpi=dpi)
for model, summary in model_summaries.items():
    if 'mean_absolute_error/kinetic_energy_density' in summary:
        plt.plot(summary['learning_rate']['simple_value'].index/model_params[model]['fit_kwargs']['epochs'], summary['mean_absolute_error/kinetic_energy_density'], label=model_labels[model])
    else:
        plt.plot([], [], label='')
plt.yscale('log')
plt.grid()
plt.legend()
plt.ylabel('mean absolute error / hartree')
plt.xlabel('training progress')
plt.savefig(os.path.join(figures_dir, 'kinetic_energy_density.eps'), format='eps', bbox_inches='tight')
plt.show()

### Model Structure